<a href="https://colab.research.google.com/github/hfelizzola/Curso-Ciencia-de-Datos-con-Python/blob/main/tareas/tarea1_limpieza_analisis/caso_retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Caso aplicado. Ventas omnicanal y devoluciones en una cadena retail — NumPy & Pandas

**Curso:** Ciencia de Datos con Python  
**Actividad:** Caso aplicado en 2 entregables  
**Entrega:** Notebook con código + respuestas en Markdown

---

> **Instrucción:** Este notebook está diseñado como *template*.  
> Encontrarás celdas con `TODO` y espacios en blanco para completar.


## Contexto del caso

Una cadena retail (electrónica/hogar) vende por **tienda**, **online** y **distribuidores** en varias ciudades de Colombia.  
El área comercial quiere entender:

- ¿Qué **canales**, **ciudades** y **categorías** generan más ventas?
- ¿Qué productos tienen **alto ingreso** pero **bajo margen**?
- ¿Dónde se concentran las **devoluciones** y por qué?

### El reto
Los datos vienen de varias fuentes:

- Fechas en formatos mixtos
- Números como texto (dinero con símbolos / separadores)
- Texto inconsistente (espacios, mayúsculas, tildes)
- Duplicados
- Faltantes (missing values)
- Valores fuera de rango (unidades negativas, descuentos imposibles)
- Llaves mal formadas (IDs y SKU inconsistentes)

---

## Fuentes de datos (archivos)

Trabajaremos con 4 CSV:

1. `ventas_raw.csv` (transaccional)
2. `clientes_raw.csv` (maestro de clientes)
3. `catalogo_raw.csv` (maestro de productos)
4. `devoluciones_raw.csv` (eventos de devolución)

> El docente te entregará estos archivos (Repo y aula virtual).


## Reglas del trabajo

- Mantén una copia `*_raw` y crea versiones `*_clean` de cada tabla.
- Documenta cada decisión (por qué imputas, por qué eliminas, etc.).
- Usa **pandas/NumPy** (evita `for` para cálculos, salvo que sea estrictamente necesario).
- Siempre valida con: `.info()`, `.isna().sum()`, `.duplicated().sum()`, y conteos (`value_counts`).

---

## Entregables

### ✅ Entregable 1 (Preparación)
Limpieza, preparación, exploración básica e ingeniería de características para dejar datos listos para análisis.

### ✅ Entregable 2 (Análisis)
Análisis con `merge`, `groupby/agg`, `pivot_table` y visualización básica para generar insights.

> Se entregar un notebook con lo avanzado en cada fase. El entregable 2 debe contener lo realizado en el entregable 1. Debe asegurarse que todo el notebook pueda correr sin errores.

---

## Checklist de calidad (recomendado)

Incluye al final de cada entregable:

- % faltantes por columna (antes y después)
- # duplicados encontrados y eliminados
- # registros removidos por reglas
- verificación de tipos (fechas, numéricos, categorías)


# 0. Configuración

In [ ]:
# =========================================
# 0. Configuración
# =========================================
import numpy as np
import pandas as pd

np.random.seed(42)

pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 140)

print("✅ Entorno listo | NumPy:", np.__version__, "| pandas:", pd.__version__)


In [ ]:
# Cargar google drive
from google.colab import drive
drive.mount("/content/drive")
# Configurar ruta de trabajo
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks") # Modifique de acuerdo a la ubicación de su carpeta de trabajo
print("✅ Ruta de trabajo:", os.getcwd())

# 1. Carga de datos

## 1.1 Cargar los CSV

**Objetivo:** cargar las 4 fuentes en DataFrames:  
`ventas_raw`, `clientes_raw`, `catalogo_raw`, `devoluciones_raw`

> Recomendación: usa rutas relativas (misma carpeta del notebook) o ajusta `DATA_DIR`.


In [ ]:
# TODO: Ajusta esta ruta si es necesario

# TODO: Carga los archivos

# TODO: Muestra una vista rápida (head) de cada uno


## 1.2 Inspección inicial (diagnóstico)

Completa:

1. Dimensiones (`shape`) de cada tabla  
2. Tipos y nulos (`info`)  
3. Resumen estadístico (`describe`)  
4. Conteos de categorías clave (`value_counts`) donde aplique (ciudad, canal, categoría, motivo devolución)

> **Meta:** identificar los principales problemas antes de limpiar.


In [ ]:
# TODO: Inspección de ventas_raw
# display(ventas_raw.head())


# TODO: Repite para clientes_raw, catalogo_raw, devoluciones_raw


### ✍️ Preguntas (responde en Markdown)

1. ¿Qué columnas tienen más valores faltantes en `ventas_raw`?  
   **Respuesta:**

2. Menciona 3 inconsistencias de texto que observes (ej. ciudad/canal).  
   **Respuesta:**

3. ¿Hay indicios de números como texto? Da 2 ejemplos de columnas.  
   **Respuesta:**


# 2. Entregable 1 — Limpieza, preparación e ingeniería de características

## Objetivo
Construir datasets limpios:

- `ventas_clean`
- `clientes_clean`
- `catalogo_clean`
- `devoluciones_clean` (opcional, pero recomendado)

Y un dataset maestro listo para análisis:

- `ventas_model` = `ventas_clean` enriquecido con catálogo y clientes (vía `merge`)


In [ ]:
# TODO: crea copia de todas las tablas
# ventas = ventas_raw.copy()



## 2.2 Normalización de texto (string cleaning)

Aplica (según columnas):

- `str.strip()` para espacios
- `str.lower()` para estandarizar
- reemplazos (ej. Bogota → Bogotá, Online → online, etc.)
- normalización de canales (ej. web → online, store → tienda)

**Sugerencia de columnas:** `ciudad`, `canal`, `producto`, `categoria_txt`, `metodo_pago`, `motivo`

> **Resultado esperado:** categorías consistentes para análisis.


In [ ]:
# TODO: normaliza texto en ventas (ciudad/canal/producto/categoría/método pago)
# ventas["ciudad"] = ventas["ciudad"].astype("string").str.strip()

# TODO: crea un diccionario de mapeo para canal (si aplica)
# canal_map = {"web":"online", "online":"online", "tienda":"tienda", "store":"tienda", "distribuidor":"distribuidor"}
# ventas["canal_norm"] = ventas["canal"].replace(canal_map)

# TODO: valida con value_counts


## 2.3 Conversión de fechas (formatos mixtos)

Convierte las columnas de fecha a `datetime` usando `pd.to_datetime(..., errors="coerce")`.

- `ventas`: `fecha`
- `clientes`: `fecha_registro`
- `devoluciones`: `fecha_devolucion`

**Entrega:**
- Reporta cuántas fechas quedaron como `NaT` y qué harás con ellas.


In [ ]:
# TODO: convertir fechas

# TODO: diagnosticar NaT


## 2.4 Conversión de numéricos que vienen como texto (dinero, porcentajes)

Problemas típicos:
- `"$ 1.200.000"` / `"1,200,000"` / `"1200000"`
- `"10%"`

**Sugerencia de estrategia:**
1. Convertir a string y limpiar símbolos (`$`, `%`, espacios)
2. Quitar separadores de miles (`.` y `,`) con cuidado
3. Convertir con `pd.to_numeric(..., errors="coerce")`

**Columnas típicas en `ventas`:**
- `precio_unitario`, `descuento`, `costo_unitario`, `unidades`

> Documenta tus supuestos (ej. descuento como porcentaje vs proporción).


In [ ]:
# TODO: ejemplo de limpieza de una columna tipo dinero


# TODO: unidades a numérico


# TODO: descuento: decide si es porcentaje ("10%") o decimal (0.10)


## 2.5 Missing values (faltantes): diagnóstico y tratamiento

Decide para cada columna si:

- **Imputas** (mediana/moda/0/regla de negocio)
- **Eliminas** filas (`dropna(subset=...)`)
- **Dejas** como faltante (si no afecta el análisis)

> Importante: justifica cada decisión.


In [ ]:
# TODO: diagnosticar faltantes

# TODO: imputación ejemplo (solo si aplica)

# TODO: eliminación ejemplo (solo si aplica)
# ventas = ventas.dropna(subset=["fecha_dt", "sku"])


## 2.6 Duplicados

Identifica duplicados y decide cómo tratarlos:

- Duplicados exactos: `duplicated().sum()`
- Duplicados por llave de negocio (ej. `id_factura` + `sku` + `fecha_dt`)

> Entrega: reporta cuántos duplicados eliminaste.


In [ ]:
# TODO: duplicados exactos


# TODO: duplicados por llave

# TODO: elimina según tu criterio


## 2.7 Reglas de negocio y outliers

Ejemplos (ajusta según el caso):

- `unidades <= 0` → revisar / eliminar
- `descuento < 0` o `descuento > 0.7` → revisar
- ventas canceladas → excluir del análisis
- costo unitario faltante → imputar por mediana por categoría (opcional)

> Documenta las reglas que aplicaste y el impacto en el # de filas.


In [ ]:
# TODO: aplica reglas de negocio (ejemplos)



## 2.8 Ingeniería de características (feature engineering)

Crea variables nuevas (mínimo 6), por ejemplo:

- `venta_bruta = unidades * precio_cop`
- `venta_neta = venta_bruta * (1 - descuento)`
- `costo_total = unidades * costo_unitario`
- `margen = venta_neta - costo_total`
- `margen_pct = margen / venta_neta`
- `mes`, `trimestre`, `dia_semana`
- `es_online` (0/1)
- `rango_descuento` con `pd.cut`
- `ticket_categoria` con `pd.qcut`

> Si una variable requiere supuestos, escríbelos.


In [ ]:
# TODO: crea features


# TODO: features de fecha

# TODO: segmentación con qcut/cut


### ✍️ Preguntas Entregable 1 (responde en Markdown)

1. ¿Qué decisiones de imputación tomaste y por qué?  
   **Respuesta:**

2. ¿Qué reglas de negocio aplicaste para limpiar outliers / registros inválidos?  
   **Respuesta:**

3. Lista tus features creadas y para qué sirven (mínimo 6).  
   **Respuesta:**

4. ¿Qué validaciones hiciste para asegurar consistencia del dataset final?  
   **Respuesta:**


## 2.9 Exportar datasets limpios (opcional)

Guarda tus resultados como CSV:

- `ventas_clean.csv`
- `clientes_clean.csv`
- `catalogo_clean.csv`
- `devoluciones_clean.csv`

> Solo si el docente lo solicita.


In [ ]:
# TODO: exportar (si aplica)
# ventas.to_csv("ventas_clean.csv", index=False)


# 3. Entregable 2 — Análisis (merge, groupby, pivot) + visualización básica

## Objetivo
Construir un mini-reporte con **tablas** y **gráficos** (básicos) para generar insights accionables.

**Se espera:**
- `merge` (enriquecer ventas con clientes y catálogo)
- `groupby().agg()` (KPIs)
- `pivot_table()` (tablas tipo Excel)
- 2–3 visualizaciones con Matplotlib (estilo simple)


In [ ]:
# TODO: merge con catálogo

# TODO: merge con clientes

# TODO: diagnóstico de joins
# ventas_model.isna().sum().sort_values(ascending=False).head(15)


## 3.2 KPIs con `groupby` + `agg`

Calcula (al menos):

- Ventas totales (`venta_neta`) por **canal**
- Ventas y margen por **ciudad**
- Ticket promedio por canal (promedio de `venta_neta`)
- Top 10 SKUs por ventas y por margen
- Productos con alta venta pero bajo margen (criterio definido por ti)

> Presenta resultados ordenados con `sort_values`.


In [ ]:
# TODO: groupby por canal



## 3.3 `pivot_table` (tablas dinámicas)

Construye al menos 2 pivots:

1. `ciudad × canal` con `venta_neta` (suma)  
2. `categoria × canal` con `venta_neta` (suma) o `margen` (suma)

Agrega totales por fila/columna si es útil.


In [ ]:
# TODO: pivot ciudad x canal (ventas)



## 3.4 Visualización básica (Matplotlib)

Crea 2–3 gráficos simples:

- Barras: ventas por canal
- Barras: margen por categoría (top 8)
- Línea: ventas por mes (por canal) *(si tienes mes)*

> No te preocupes por estética avanzada; eso se trabajará en una sesión futura.


In [ ]:
# TODO: visualización
import matplotlib.pyplot as plt

# Ejemplo (completa con tu tabla kpi_canal):
# kpi_canal["ventas"].plot(kind="bar")
# plt.title("Ventas por canal")
# plt.ylabel("Venta neta (COP)")
# plt.show()


### ✍️ Preguntas Entregable 2 (responde en Markdown)

1. ¿Qué canal genera más ventas? ¿También genera más margen?  
   **Respuesta:**

2. ¿Qué ciudad es líder en ventas y qué hipótesis de negocio planteas?  
   **Respuesta:**

3. Identifica un caso de “alto ingreso pero bajo margen” (producto/categoría).  
   **Respuesta:**

4. Si analizaste devoluciones: ¿dónde se concentra la tasa de devolución y por qué crees que ocurre?  
   **Respuesta:**

5. Propón 2 acciones recomendadas para la empresa basadas en tus hallazgos.  
   **Respuesta:**


# 4. Cierre

## Autoevaluación rápida
Marca con ✅ cuando lo completes:

- [ ] Convertí fechas y validé `NaT`
- [ ] Convertí dinero/porcentajes a numérico
- [ ] Traté faltantes y documenté decisiones
- [ ] Eliminé duplicados con criterio
- [ ] Creé al menos 6 features
- [ ] Integré datos con `merge` y validé matches
- [ ] Construí KPIs con `groupby().agg()`
- [ ] Construí 2 pivot tables
- [ ] Hice 2–3 gráficos básicos
- [ ] Escribí insights y recomendaciones

---
